<a href="https://colab.research.google.com/github/Hari25483/Nail-Disease-Analysis/blob/main/Nail_severity_Mask_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # Update CUDA for TF 2.5
# !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
# !dpkg -i libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
# # Check if package has been installed
# !ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so.*
# Upgrade Tensorflow
# !pip install tensorflow==2.5.0 -f https://storage.googleapis.com/tf-debians/index.html


In [ ]:
import tensorflow
assert tensorflow.__version__ == '2.5.0'

In [ ]:
# !wget https://pysource.com/extra_files/maskrcnn_colab_demo_commit_17.zip
# !unzip maskrcnn_colab_demo_commit_17.zip
# import sys
# sys.path.append("/content/drive/MyDrive/mask_nail/maskrcnn_colab/mrcnn_demo")
# from m_rcnn import *
# %matplotlib inline

In [ ]:
!nvdia-smi

## **4. Detection (test your model on a random image)**

In [ ]:
# Test on a random image
# %cd /content/drive/MyDrive/mask_nail/
#thanujan

In [ ]:
# ROOT_DIR = "/content/drive/MyDrive/mask_nail/maskrcnn_colab/mrcnn_demo"
# %cd mrcnn_demo/

In [ ]:
import sys
# sys.path.append("/content/drive/MyDrive/mask_nail/maskrcnn_colab/mrcnn_demo")
sys.path.append("./maskrcnn_colab/mrcnn_demo")
from m_rcnn import *
from visualize import random_colors, get_mask_contours, draw_mask
%matplotlib inline

In [ ]:
# %cd /content/

In [ ]:
%%time
# test_model, inference_config = load_inference_model(1, "/content/drive/MyDrive/mask_nail/maskrcnn_colab/logs/object20230122T1637/mask_rcnn_object_0005.h5")
test_model, inference_config = load_inference_model(1, "./mask_rcnn_object_0005.h5")


In [ ]:
inference_config.display()

In [ ]:
# Load Image
# img = cv2.imread("/content/funger.jpeg")
img = cv2.imread("./test.jpg")

# test_model, inference_config = load_inference_model(1, "/content/drive/MyDrive/mask_nail/maskrcnn_colab/logs/object20230118T0240/mask_rcnn_object_0005.h5")
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)



In [ ]:
%%time
# Detect results
r = test_model.detect([image])[0]

In [ ]:
# from google.colab.patches import cv2_imshow
# Get Coordinates and show it on the image

print(r["masks"].shape)
object_count = len(r["class_ids"])
colors = random_colors(object_count)
colors = random_colors(object_count)
for i in range(object_count):
    # 1. Mask
    mask = r["masks"][:, :, i]
    contours = get_mask_contours(mask)
    for cnt in contours:
        cv2.polylines(img, [cnt], True, colors[i], 2)
        output = draw_mask(img, [cnt], colors[i])

cv2.imshow("", output)
cv2.waitKey(0)

# closing all open windows
cv2.destroyAllWindows()

In [ ]:
#!susimport cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Load Image
img = cv2.imread("./b (30).jpg")


# test_model, inference_config = load_inference_model(1, "./maskrcnn_colab/logs/object20230122T1637/mask_rcnn_object_0005.h5")
image_a = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Detect results
r = test_model.detect([image_a])[0]
colors = random_colors(80)
# Get Coordinates and show it on the image
object_count = len(r["class_ids"])
for i in range(object_count):
    # 1. Mask
    mask = r["masks"][:, :, i]
    contours = get_mask_contours(mask)
    for cnt in contours:
        # cv2.polylines(img_a, [cnt], True, colors[i], 2)
        img_a = draw_mask(img, [cnt], colors[i])
# cv2.imshow(img)

# img = cv2.imread("/content/b (10).jpg")
pts = cnt

## (1) Crop the bounding rect
rect = cv2.boundingRect(pts)
x,y,w,h = rect
croped = img[y:y+h, x:x+w].copy()

## (2) make mask
pts = pts - pts.min(axis=0)

mask = np.zeros(croped.shape[:2], np.uint8)
cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

## (3) do bit-op
dst = cv2.bitwise_and(croped, croped, mask=mask)

## (4) add the white background
## (4) add the white background
bg = np.ones_like(croped, np.uint8)*255
cv2.bitwise_not(bg,bg, mask=mask)
image = bg+ dst


cv2.imshow("croped", croped)
cv2.imshow("mask", mask)
cv2.imshow("dst", dst)
cv2.imshow("image", image)
cv2.waitKey(0)

# closing all open windows
cv2.destroyAllWindows()

brown_range = [(0, 0, 0), (170, 170, 170)]

2
# Define the replacement colors for green and brown
brown_color = (139, 69, 19)

# Convert the image to a numpy array
image_array = np.array(image)


# Replace the pixels in the brown range with the brown color
brown_mask = cv2.inRange(image_array, brown_range[0], brown_range[1])
image_array[np.where(brown_mask)] = brown_color

# Convert the rest of the pixels to white
# white_mask = cv2.bitwise_not(brown_mask)
# image_array[np.where(white_mask)] = (255, 255, 255)
white = np.array([255, 255, 255])   # white color value
brown = np.array([139, 69, 19])      # brown color value
green = np.array([0, 255, 0])       # green color value

brown_mask = np.all(image_array == brown, axis=-1)
non_white_brown_mask = np.logical_not(np.all(image_array == white, axis=-1) | brown_mask)
image_array[non_white_brown_mask] = green  # modify non-white and non-brown pixels to green

# Convert the numpy array back to an image
segmented_image = Image.fromarray(image_array)

plt.imshow(segmented_image)
plt.show()

# Convert the image to a NumPy array
img_array = np.array(segmented_image)
img_array_a = np.array(image)

# Get the dimensions of the image
height, width, channels = img_array.shape
height_a, width_a, channels_a = img_array_a.shape
output

# Define the RGB color you want to find the area of distribution for
rgb_color = [139, 69, 19] # for red
white = [255, 255, 255]

# Loop through the pixels of the image and count the number of pixels with the desired RGB color
counter = 0
for i in range(height):
    for j in range(width):
        if np.array_equal(img_array[i,j], rgb_color):
            counter += 1

counter_a = 0
for i in range(height_a):
    for j in range(width_a):
        if not np.array_equal(img_array_a[i,j], white):
            counter_a += 1

print(f"The area of distribution of the RGB color {rgb_color} is: {counter}")
print(f"The distribution of non-white pixels in the image is: {counter_a}")
print(f"percentage: {(counter/counter_a)*100}")
print('shape1',img_array.shape)
print('shape2',img_array_a.shape)

In [ ]:
import matplotlib.pyplot as plt
sub=counter_a-counter
# Define the values and labels for the pie chart
values = [sub,counter]
labels = ['Green', 'Brown']
colors = ['lime', 'saddlebrown']


# Create the pie chart with percentage values
plt.pie(values, labels=labels, autopct='%1.1f%%',colors=colors)

# Set the title of the chart
plt.title('Pie Chart')

# Show the chart
plt.show()

# yellow nail

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Load Image
img = cv2.imread("./n.jpg")


# test_model, inference_config = load_inference_model(1, "./maskrcnn_colab/logs/object20230122T1637/mask_rcnn_object_0005.h5")
image_a = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Detect results
r = test_model.detect([image_a])[0]
colors = random_colors(80)
# Get Coordinates and show it on the image
object_count = len(r["class_ids"])
for i in range(object_count):
    # 1. Mask
    mask = r["masks"][:, :, i]
    contours = get_mask_contours(mask)
    for cnt in contours:
        # cv2.polylines(img_a, [cnt], True, colors[i], 2)
        img_a = draw_mask(img, [cnt], colors[i])
# cv2.imshow(img)

# img = cv2.imread("/content/b (10).jpg")
pts = cnt

## (1) Crop the bounding rect
rect = cv2.boundingRect(pts)
x,y,w,h = rect
croped = img[y:y+h, x:x+w].copy()

## (2) make mask
pts = pts - pts.min(axis=0)

mask = np.zeros(croped.shape[:2], np.uint8)
cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

## (3) do bit-op
dst = cv2.bitwise_and(croped, croped, mask=mask)

## (4) add the white background
## (4) add the white background
bg = np.ones_like(croped, np.uint8)*255
cv2.bitwise_not(bg,bg, mask=mask)
image = bg+ dst


cv2.imshow("croped", croped)
cv2.imshow("mask", mask)
cv2.imshow("dst", dst)
cv2.imshow("image", image)
cv2.waitKey(0)

# closing all open windows
cv2.destroyAllWindows()

brown_range = [(100, 10, 100), (250, 250, 200)]


# Define the replacement colors for green and brown
brown_color = (139, 69, 19)

# Convert the image to a numpy array
image_array = np.array(image)


# Replace the pixels in the brown range with the brown color
brown_mask = cv2.inRange(image_array, brown_range[0], brown_range[1])
image_array[np.where(brown_mask)] = brown_color

# Convert the rest of the pixels to white
# white_mask = cv2.bitwise_not(brown_mask)
# image_array[np.where(white_mask)] = (255, 255, 255)
white = np.array([255, 255, 255])   # white color value
brown = np.array([139, 69, 19])      # brown color value
green = np.array([0, 255, 0])       # green color value

brown_mask = np.all(image_array == brown, axis=-1)
non_white_brown_mask = np.logical_not(np.all(image_array == white, axis=-1) | brown_mask)
image_array[non_white_brown_mask] = green  # modify non-white and non-brown pixels to green

# Convert the numpy array back to an image
segmented_image = Image.fromarray(image_array)

plt.imshow(segmented_image)
plt.show()

# Convert the image to a NumPy array
img_array = np.array(segmented_image)
img_array_a = np.array(image)

# Get the dimensions of the image
height, width, channels = img_array.shape
height_a, width_a, channels_a = img_array_a.shape


# Define the RGB color you want to find the area of distribution for
rgb_color = [139, 69, 19] # for red
white = [255, 255, 255]

# Loop through the pixels of the image and count the number of pixels with the desired RGB color
counter = 0
for i in range(height):
    for j in range(width):
        if np.array_equal(img_array[i,j], rgb_color):
            counter += 1

counter_a = 0
for i in range(height_a):
    for j in range(width_a):
        if not np.array_equal(img_array_a[i,j], white):
            counter_a += 1

print(f"The area of distribution of the RGB color {rgb_color} is: {counter}")
print(f"The distribution of non-white pixels in the image is: {counter_a}")
print(f"percentage: {(counter/counter_a)*100}")
print('shape1',img_array.shape)
print('shape2',img_array_a.shape)

In [ ]:
import matplotlib.pyplot as plt
sub=counter_a-counter
# Define the values and labels for the pie chart
values = [sub,counter]
labels = ['Green', 'Brown']
colors =  ['lime', 'saddlebrown']

# Create the pie chart with percentage values
plt.pie(values, labels=labels, autopct='%1.1f%%',colors=colors)

# Set the title of the chart
plt.title('Pie Chart')

# Show the chart
plt.show()

# white nail

In [ ]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image
# Load Image
img = cv2.imread("./n.jpg")


# test_model, inference_config = load_inference_model(1, "/content/drive/MyDrive/mask_nail/maskrcnn_colab/logs/object20230122T1637/mask_rcnn_object_0005.h5")
image_a = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Detect results
r = test_model.detect([image_a])[0]
colors = random_colors(80)
# Get Coordinates and show it on the image
object_count = len(r["class_ids"])
for i in range(object_count):
    # 1. Mask
    mask = r["masks"][:, :, i]
    contours = get_mask_contours(mask)
    for cnt in contours:
        # cv2.polylines(img_a, [cnt], True, colors[i], 2)
        img_a = draw_mask(img, [cnt], colors[i])
# cv2_imshow(img)

# img = cv2.imread("/content/b (10).jpg")
pts = cnt

## (1) Crop the bounding rect
rect = cv2.boundingRect(pts)
x,y,w,h = rect
croped = img[y:y+h, x:x+w].copy()

## (2) make mask
pts = pts - pts.min(axis=0)

mask = np.zeros(croped.shape[:2], np.uint8)
cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

## (3) do bit-op
dst = cv2.bitwise_and(croped, croped, mask=mask)

## (4) add the white background
## (4) add the white background
bg = np.ones_like(croped, np.uint8)*255
cv2.bitwise_not(bg,bg, mask=mask)
image = bg+ dst


cv2.imshow("croped", croped)
cv2.imshow("mask", mask)
cv2.imshow("dst", dst)
cv2.imshow("image", image)
cv2.waitKey(0)

# closing all open windows
cv2.destroyAllWindows()

brown_range = [(160, 150, 160), (250, 250, 250)]


# Define the replacement colors for green and brown
brown_color = (139, 69, 19)

# Convert the image to a numpy array
image_array = np.array(image)


# Replace the pixels in the brown range with the brown color
brown_mask = cv2.inRange(image_array, brown_range[0], brown_range[1])
image_array[np.where(brown_mask)] = brown_color

# Convert the rest of the pixels to white
# white_mask = cv2.bitwise_not(brown_mask)
# image_array[np.where(white_mask)] = (255, 255, 255)
white = np.array([255, 255, 255])   # white color value
brown = np.array([139, 69, 19])      # brown color value
green = np.array([0, 255, 0])       # green color value

brown_mask = np.all(image_array == brown, axis=-1)
non_white_brown_mask = np.logical_not(np.all(image_array == white, axis=-1) | brown_mask)
image_array[non_white_brown_mask] = green  # modify non-white and non-brown pixels to green

# Convert the numpy array back to an image
segmented_image = Image.fromarray(image_array)

plt.imshow(segmented_image)
plt.show()

# Convert the image to a NumPy array
img_array = np.array(segmented_image)
img_array_a = np.array(image)

# Get the dimensions of the image
height, width, channels = img_array.shape
height_a, width_a, channels_a = img_array_a.shape


# Define the RGB color you want to find the area of distribution for
rgb_color = [139, 69, 19] # for red
white = [255, 255, 255]

# Loop through the pixels of the image and count the number of pixels with the desired RGB color
counter = 0
for i in range(height):
    for j in range(width):
        if np.array_equal(img_array[i,j], rgb_color):
            counter += 1

counter_a = 0
for i in range(height_a):
    for j in range(width_a):
        if not np.array_equal(img_array_a[i,j], white):
            counter_a += 1

print(f"The area of distribution of the RGB color {rgb_color} is: {counter}")
print(f"The distribution of non-white pixels in the image is: {counter_a}")
print(f"percentage: {(counter/counter_a)*100}")
print('shape1',img_array.shape)
print('shape2',img_array_a.shape)

In [ ]:
#  import matplotlib.pyplot as plt
sub=counter_a-counter
# Define the values and labels for the pie chart
values = [sub,counter]
labels = ['Green', 'Brown']
colors =  ['lime', 'saddlebrown']

# Create the pie chart with percentage values
plt.pie(values, labels=labels, autopct='%1.1f%%',colors=colors)

# Set the title of the chart
plt.title('Pie Chart')

# Show the chart
plt.show()


In [ ]:
test_model, inference_config = load_inference_model(1, "./maskrcnn_colab/logs/object20230122T1637/mask_rcnn_object_0005.h5")


In [ ]:
# !pip install -U flask-cors
# !pip install flask-ngrok
# !pip uninstall -y flask-ngrok
# !pip install flask-ngrok
from flask import Flask,request, jsonify, g
from flask_ngrok import run_with_ngrok

import pickle
from flask_cors import CORS, cross_origin
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from PIL import Image
import tensorflow as tf

from tensorflow.python.keras.backend import set_session

In [ ]:
# !ngrok authtoken 2ObJjzVK6m5Pnb0A3PEnKtFTkTL_56ACd4yKvSQPcS6UPNtBk

# tf.compat.v1.disable_eager_execution()

# hello = tf.constant('Hello, TensorFlow!')

# session = tf.compat.v1.Session()
import os

app = Flask(__name__)
# sess = tf.Session()
CORS(app)
run_with_ngrok(app)


test_model, inference_config = load_inference_model(1, "./maskrcnn_colab/logs/object20230122T1637/mask_rcnn_object_0005.h5")



# class Inference:
#   global graph
#   __model = None
#   __instance = None

#   def __new__(cls,arg):
#     if cls.__instance is None:
#       cls.__instance = super().__new__(cls)
#     return cls.__instance

#   def getPrediction(self, img):
#     with graph.as_default():
#       try:
#         return self.__model.detect([img], verbose=0)
#       except Exception as e:
#         print(e)

#   def __init__(self, path):
#     if self.__model == None:
#       print("model set")
#       model, inference =  load_inference_model(1,path)
#       self.__model = model



# def load_model():
#   global test_model
#   global inference_config
#   global graph
#   # graph = tf.compat.v1.get_default_graph()

#   with graph.as_default():
#     test_model = Inference( "./maskrcnn_colab/logs/object20230122T1637/mask_rcnn_object_0005.h5")


# @app.before_first_request
# def before_first_request():
#     load_model()


@app.route('/')
def index():
    return jsonify({"Hello":"Welcome"})

@app.route('/upload_image', methods=['POST'])
def upload_image():
  global test_model
  global graph
  print(request.files)
  print(request.form)
  #return jsonify({"Hello":"Welcome"})
  if 'image' in request.files:
    image = request.files['image']
    Nail_disease = request.form.get('nail_disease')
    print(Nail_disease)
    image.save(image.filename)
    print(image.filename)
    # Load Image
    # Example file name
    filename =image.filename
    # Get the current working directory
    cwd = os.getcwd()
    # Join the file name with the current working directory to get the full file path
    file_path = os.path.join(cwd, filename)

    print(file_path)
    if (Nail_disease=="black nail"):
        brown_range = [(0, 0, 0), (150, 140, 150)]
    elif (Nail_disease=='yellow nail'):
        brown_range = [(100, 10, 100), (255, 250, 200)]
    elif (Nail_disease=='white spot'):
        brown_range = [(160, 150, 160), (250, 250, 250)]
    else:
        return (jsonify("Wrong disease"))


    img = cv2.imread(file_path)
    print(img.shape)
    # imageFileName = secure_filename(image.filename)
    # image.save('./images/' + imageFileName)
    # image = cv2.imread('./images/' + imageFileName)
    # with session.as_default():
    #     with session.graph.as_default():
    # test_model, inference_config = load_inference_model(1, "/content/drive/MyDrive/mask_nail/maskrcnn_colab/logs/object20230122T1637/mask_rcnn_object_0005.h5")
    test_model, inference_config = load_inference_model(1, "./maskrcnn_colab/logs/object20230122T1637/mask_rcnn_object_0005.h5")

    image_a = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    result = test_model.detect([image_a], verbose=0)
    r = result[0]


    colors = random_colors(80)
    # Get Coordinates and show it on the image

    object_count = len(r["class_ids"])
    for i in range(object_count):
        # 1. Mask
        mask = r["masks"][:, :, i]
        contours = get_mask_contours(mask)
        for cnt in contours:
            cv2.polylines(image_a, [cnt], True, colors[i], 2)
              # img_a = draw_mask(img, [cnt], colors[i])

    # cv2_imshow(img)

    # img = cv2.imread("/content/b (10).jpg")
    pts = cnt

    ## (1) Crop the bounding rect
    rect = cv2.boundingRect(pts)
    x,y,w,h = rect
    croped = img[y:y+h, x:x+w].copy()

    ## (2) make mask
    pts = pts - pts.min(axis=0)

    mask = np.zeros(croped.shape[:2], np.uint8)
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

    ## (3) do bit-op
    dst = cv2.bitwise_and(croped, croped, mask=mask)

    ## (4) add the white background
    ## (4) add the white background
    bg = np.ones_like(croped, np.uint8)*255
    cv2.bitwise_not(bg,bg, mask=mask)
    image = bg+ dst


    # cv2.imshow(croped)
    # cv2.imshow(mask)
    # cv2.imshow(dst)
    # cv2.imshow(image)


    # Define the replacement colors for green and brown
    brown_color = (139, 69, 19)

    # Convert the image to a numpy array
    image_array = np.array(image)


    # Replace the pixels in the brown range with the brown color
    brown_mask = cv2.inRange(image_array, brown_range[0], brown_range[1])
    image_array[np.where(brown_mask)] = brown_color

    # Convert the rest of the pixels to white
    white_mask = cv2.bitwise_not(brown_mask)
    image_array[np.where(white_mask)] = (255, 255, 255)

    # Convert the numpy array back to an image
    segmented_image = Image.fromarray(image_array)

    # plt.imshow(segmented_image)
    # plt.show()

    # Convert the image to a NumPy array
    img_array = np.array(segmented_image)
    img_array_a = np.array(image)

    # Get the dimensions of the image
    height, width, channels = img_array.shape
    height_a, width_a, channels_a = img_array_a.shape


    # Define the RGB color you want to find the area of distribution for
    rgb_color = [139, 69, 19] # for red
    white = [255, 255, 255]

    # Loop through the pixels of the image and count the number of pixels with the desired RGB color
    counter = 0
    for i in range(height):
      for j in range(width):
        if np.array_equal(img_array[i,j], rgb_color):
              counter += 1

    counter_a = 0
    for i in range(height_a):
      for j in range(width_a):
        if not np.array_equal(img_array_a[i,j], white):
          counter_a += 1
    print(f"The area of distribution of the RGB color {rgb_color} is: {counter}")
    print(f"The distribution of non-white pixels in the image is: {counter_a}")
    print(f"percentage: {(counter/counter_a)*100}")
    print("hello")
    # pred('/content/b.PNG')
    return(jsonify(math.ceil((counter/counter_a)*100)))


app.run()
